# Word Prediction using Markov Model

This notebook makes use of Markov model for word prediction. Specifically 2nd order Markov model is deployed here for next word prediction. As an example of the Markov chain, an attempt is made to generate a new song lyrics from a bunch of Eminem song lyrics.

In [ ]:
# Preamble
import string
import numpy as np

In [ ]:
# Path of the text file containing the training data
training_data_file = 'F:/eminem_songs_lyrics.txt'

## Training

### Helper functions

In [ ]:
def remove_punctuation(sentence):
    return sentence.translate(str.maketrans('','', string.punctuation))

In [ ]:
def add2dict(dictionary, key, value):
    if key not in dictionary:
        dictionary[key] = []
    dictionary[key].append(value)

In [ ]:
def list2probabilitydict(given_list):
    probability_dict = {}
    given_list_length = len(given_list)
    for item in given_list:
        probability_dict[item] = probability_dict.get(item, 0) + 1
    for key, value in probability_dict.items():
        probability_dict[key] = value / given_list_length
    return probability_dict

In [ ]:
initial_word = {}
second_word = {}
transitions = {}

### Training function

In [ ]:
# Trains a Markov model based on the data in training_data_file
def train_markov_model():
    for line in open(training_data_file,encoding='utf-8'):
        tokens = remove_punctuation(line.rstrip().lower()).split()
        tokens_length = len(tokens)
        for i in range(tokens_length):
            token = tokens[i]
            if i == 0:
                initial_word[token] = initial_word.get(token, 0) + 1
            else:
                prev_token = tokens[i - 1]
                if i == tokens_length - 1:
                    add2dict(transitions, (prev_token, token), 'END')
                if i == 1:
                    add2dict(second_word, prev_token, token)
                else:
                    prev_prev_token = tokens[i - 2]
                    add2dict(transitions, (prev_prev_token, prev_token), token)

    # Normalize the distributions
    initial_word_total = sum(initial_word.values())
    for key, value in initial_word.items():
        initial_word[key] = value / initial_word_total

    for prev_word, next_word_list in second_word.items():
        second_word[prev_word] = list2probabilitydict(next_word_list)

    for word_pair, next_word_list in transitions.items():
        transitions[word_pair] = list2probabilitydict(next_word_list)

    print('Training successful.')

In [ ]:
train_markov_model()

Training successful.


In [ ]:
print('second_word',second_word)

second_word {'ive': {'been': 0.8333333333333334, 'got': 0.16666666666666666}, 'been': {'a': 1.0}, 'all': {'my': 0.875, 'the': 0.125}, 'well': {'little': 0.6666666666666666, 'to': 0.16666666666666666, 'thats': 0.16666666666666666}, 'if': {'all': 0.6666666666666666, 'that': 0.16666666666666666, 'i': 0.16666666666666666}, 'truth': {'and': 1.0}, 'so': {'let': 0.2857142857142857, 'shes': 0.07142857142857142, 'i': 0.14285714285714285, 'the': 0.07142857142857142, 'here': 0.07142857142857142, 'gay': 0.07142857142857142, 'youll': 0.07142857142857142, 'ray': 0.07142857142857142, 'while': 0.07142857142857142, 'this': 0.07142857142857142}, 'hes': {'comin': 0.16666666666666666, 'nervous': 0.16666666666666666, 'choking': 0.16666666666666666, 'so': 0.3333333333333333, 'grown': 0.16666666666666666}, 'sweat': {'jackets': 1.0}, 'on': {'his': 0.25, 'the': 0.5, 'im': 0.25}, 'and': {'she': 0.041666666666666664, 'guess': 0.041666666666666664, 'you': 0.08333333333333333, 'i': 0.25, 'now': 0.04166666666666666

### Generating text



In [ ]:
def sample_word(dictionary):
    p0 = np.random.random()
    cumulative = 0
    for key, value in dictionary.items():
        cumulative += value
        if p0 < cumulative:
            return key
    assert(False)

In [ ]:
number_of_sentences = 10

In [ ]:
# Function to generate sample text
def generate():
    for i in range(number_of_sentences):
        sentence = []
        # Initial word
        word0 = sample_word(initial_word)
        sentence.append(word0)
        # Second word
        word1 = sample_word(second_word[word0])
        sentence.append(word1)
        # Subsequent words untill END
        while True:
            word2 = sample_word(transitions[(word0, word1)])
            if word2 == 'END':
                break
            sentence.append(word2)
            word0 = word1
            word1 = word2
        print(' '.join(sentence))

In [ ]:
generate()

my thoughts are sporadic i act like i’m an addict
so while you’re in it try to get this one chance
oh hes too mainstream
the roof comes off
rappers are having a rough time period
the truth and my lies straight
i cant keep my lies now are falling like the rain
little gaylooking boy
my honestys brutal
oh hes too mainstream
